In [ ]:
from riotwatcher import LolWatcher, ApiError, RateLimiter
import pandas as pd

In [ ]:
riotwatcher.RateLimiter()

In [ ]:
#riot games api key
api_key = ''

#set region, init lol watcher obj
region = 'na1'
lol_watcher = LolWatcher(api_key)
me = 'Tape'

In [ ]:
lol_watcher.summoner.by_name(region, me)

In [ ]:
#return challengers
challengers = lol_watcher.league.challenger_by_queue(region, 'RANKED_SOLO_5x5')

#return grandmasters
gms = lol_watcher.league.grandmaster_by_queue(region, 'RANKED_SOLO_5x5')

#return masters
masters = lol_watcher.league.masters_by_queue(region, 'RANKED_SOLO_5x5')

In [ ]:
#list of the above objects
all_top_players = [challengers, gms, masters]

#loop through and concat all summoner Ids
summoner_ids = []
for division in all_top_players:
    for entry in division['entries']:
        summoner_ids.append(list(entry.values())[0])

In [ ]:
#now we have all summoner IDs in a list
#because my key is a development key, limited to 100 requests every 2 minutes
#i want to make a lot more queries than that

In [ ]:
#map summid to puuid
summid_to_puuid = {}
for summoner in summoner_ids:
    summid_to_puuid[summoner] =  lol_watcher.summoner.by_id(region, summoner)['puuid']

In [ ]:
#create dict to hold champion masteries for each summoner by puuid
mastery_dict = {}

#fill dict
for summoner in summoner_ids:
    #make request for champion masteries, store in variable
    masteries = lol_watcher.champion_mastery.by_summoner(region, summoner)
    #loop through, adding high mastery champs to list
    puuid = summid_to_puuid[summoner]
    mastery_dict[puuid] = []
    for mastery in masteries:
        if mastery.get('championPoints') > 100000:
            mastery_dict[puuid].append(mastery.get('championId'))

In [ ]:
#okay, now we could iterate over each summoner and grab matches. However, we could likely
#improve runtime by checking for each summoner in our list to see if they were in the game

#create list to store dict objects
data_rows = []
            
#store set of matches already looked through
matches_scanned = set()

In [ ]:
#list of features we want to record
features = ['puuid',
            'championId',
            'item0',
            'item1',
            'item2',
            'item3',
            'item4',
            'item5',
            'item6',
            'kills',
            'deaths',
            'assists',
            'totalDamageDealtToChampions',
            'role',
            'teamPosition',
            'teamId',
            'gameEndedInEarlySurrender',
            'win']

# TODO: add error 503 handling
for key, value in mastery_dict.items():
    #store matchlist for each puuid
    match_list = lol_watcher.match.matchlist_by_puuid(region, key, count = 50)
    for match in match_list:
        if match not in matches_scanned:
            #store match data in variable
            match_data = lol_watcher.match.by_id(region, match)
            #store participant information in variable to iterate over (list of dicts) if classic game
            if match_data['info']['gameMode'] == 'CLASSIC':
                player_info = match_data['info']['participants']
                #create dict of champs on team1, team2
                champions_in_game = {}
                champions_in_game[100] = []
                champions_in_game[200] = []
                for player in player_info:
                    #add champ played to dict
                    champions_in_game[player['teamId']].append(player['championId'])
                    #check to see if player in our list of masters+ players
                    if player['puuid'] in mastery_dict.keys(): 
                        #check to see if player on a high mastery champ
                        if player['championId'] in mastery_dict[player['puuid']]:
                            #get player data, store in dictionary
                            player_data = {}
                            for feature in features:
                                player_data[feature] = player[feature]
                            player_data['match_id'] = match
                            player_data['champions_in_game'] = champions_in_game
                            #append dictionary to list
                            data_rows.append(player_data)
                            
                #print out to watch progress
                #print('champion ID: ',player_data['championId'],', win:',player_data['win'])
                #print('champs in game: ',player_data['champions_in_game'])
                            
            #append match_id to matches_scanned set
            matches_scanned.add(match)



In [ ]:
#do we have info for every champion?
champs_logged = set()
for row in data_rows:
    champs_logged.add(row['championId'])
len(champs_logged)

In [ ]:
#we dooooooooo
#construct a row for each entry in the data_rows object to go into our dataframe
dataframe[0]

In [ ]:
#lets immediately put this in a database before we break it
import sqlite3

In [ ]:
conn = sqlite3.connect('match_entries.db')

In [ ]:
df = pd.DataFrame.from_dict(data_rows)

In [ ]:
df = pd.DataFrame.from_dict(dataframe)
df.to_csv('matches.csv')

In [ ]:
#backup var
data_rows2 = data_rows

In [ ]:
df["champions_in_game"][0][100]

In [ ]:
#drop where teamPosition empty
df = df[df['teamPosition'] != '']
#drop where game ended in early surrender
df = df[df['gameEndedInEarlySurrender'] == False]

In [ ]:
df[['teamId','champions_in_game']]

In [ ]:
#lets construct columns from the teamId and champions_in_game column

#new column, list of champions on player's team
df['teammates_championId'] = df.apply(lambda x: x['champions_in_game'].get(x['teamId']), axis=1)

#new column, list of enemy champions
opposite_team_dict = {100:200, 200:100}
df['opposite_team_id'] = df['teamId'].map(opposite_team_dict)
df['enemies_championId'] = df.apply(lambda x: x['champions_in_game'].get(x['opposite_team_id']), axis=1)

In [ ]:
df = df.drop(labels=["champions_in_game","opposite_team_id"], axis=1)

In [ ]:
#split list into individual columns
player_cols = ["enemies_championId", "teammates_championId"]
for col in player_cols:
    temp_df = df[col].apply(pd.Series)
    temp_df = temp_df.add_prefix(col[:-10])
    df = pd.concat([df, temp_df], axis=1)